In [1]:
from data_utils import load_rt

In [2]:
pos, neg = load_rt()

In [3]:
print(len(pos)); print(len(neg))

print(pos[0])

print(neg[0])

5331
5331
the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
simplistic , silly and tedious .


In [4]:
#find max length and pad all reviews below max length

max_len = 0

pos_and_neg = pos + neg

for pn in pos_and_neg:
    if len(pn.split()) > max_len:
        max_len = len(pn.split())
        
for i, _ in enumerate(pos):
    while len(pos[i].split()) < max_len:
        pos[i] += (" <PAD>")

for i, _ in enumerate(neg):
    while len(neg[i].split()) < max_len:
        neg[i] += (" <PAD>")
        
        
print(pos[0])
print(len(pos[0].split()))

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
59


In [5]:
#build vocab of unique words to turn into indices

pos_and_neg = pos + neg #re-define this as sequences are now padded!

vocab = set()

all_words = [w for pn in pos_and_neg for w in pn.split()]

vocab.update(all_words)

In [6]:
#build word to index dictionary

word2idx = { w:i for i,w in enumerate(vocab) }
idx2word = { i:w for i,w in enumerate(vocab) }

In [7]:
#convert words to indices and add labels, 1 for pos, 0 for neg

data = []

for p in pos:
    p_index = [word2idx[w] for w in p.split()]
    data.append([p_index, 1])
    
for n in neg:
    n_index = [word2idx[w] for w in n.split()]
    data.append([n_index, 0])
    
print(data[0])
print([idx2word[i] for i in data[0][0]])

[[18934, 20460, 12057, 17470, 9215, 8980, 18934, 2378, 5643, 7649, 17019, 13429, 17019, 13949, 5867, 7594, 850, 9215, 6537, 13307, 12890, 131, 18010, 8300, 299, 19708, 4124, 1396, 8326, 7347, 21248, 19936, 10803, 3518, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623, 11623], 1]
['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', "century's", 'new', '"', 'conan', '"', 'and', 'that', "he's", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', ',', 'jean-claud', 'van', 'damme', 'or', 'steven', 'segal', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [13]:
#define network
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super(Net, self).__init__()
    
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.gru = nn.GRU()
        
        self.logsoftmax = nn.LogSoftmax()
        
    def forward(self, x):
        #x = self.embedding(x)
        x = x.float()
        x = x.view(1, 59)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.logsoftmax(x)
        return x

In [14]:
#define network

net = Net(len(vocab), 100)

In [15]:
#set hyperparameters 

epochs = 10
batch_size = 32
learning_rate = 0.001
momentum = 0.9

In [16]:
#define loss and optimizer

import torch.optim as optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

In [27]:
from torch.autograd import Variable

print(net)

for e in range(1, epochs+1):
    
    for d in data:
        
        input_variable = Variable(torch.LongTensor(d[0]))

        if d[1] == 0:
            target_variable = Variable(torch.LongTensor([0]))
        else:
            target_variable = Variable(torch.LongTensor([1]))
            
        optimizer.zero_grad()

        fx = net(input_variable)

        loss = criterion(fx, target_variable)

        loss.backward()

        optimizer.step()
    
    print("Epoch %02d, loss = %f" % (e, loss.data[0]))
        
    correct = 0
        
    for d in data:
        
        input_variable = Variable(torch.LongTensor(d[0]))
        
        if d[1] == 0:
            target = torch.LongTensor([0])
        else:
            target = torch.LongTensor([1])
            
        output = net(input_variable)
        
        _, predicted = torch.max(output.data, 1) #for each output, get the predicted value (torch.max returns (index, value) tuple)
        
        #print(predicted)
        #print(target)
        is_correct = (predicted == target)
        #print(is_correct)
        correct += is_correct.sum()
    
    print(correct)
            

Net (
  (embedding): Embedding(21402, 100)
  (fc1): Linear (59 -> 100)
  (fc2): Linear (100 -> 2)
  (logsoftmax): LogSoftmax ()
)
Epoch 01, loss = 0.010163
5335
Epoch 02, loss = 0.010137
5335
Epoch 03, loss = 0.010137


KeyboardInterrupt: 